In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import time
import io
import re
import nltk
import numpy as np
from keras.models import Model
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Input, Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint, TensorBoard
from collections import Counter
from sklearn.model_selection import train_test_split

np.random.seed(2019)

tf.estimator package not installed.
tf.estimator package not installed.


Using TensorFlow backend.


In [27]:
# set default parameters
BATCH_SIZE = 128
NUM_EPOCHS = 1
HIDDEN_UNITS = 256
MAX_INPUT_SEQ_LENGTH = 20
MAX_TARGET_SEQ_LENGTH = 20
MAX_VOCAB_SIZE = 500
DATA_PATH ='/Users/troyxiao/Desktop/NLP-project/movie_lines.txt'
WEIGHT_FILE_PATH = 'model/word-weights.h5'

In [2]:
lines = open('/Users/troyxiao/Desktop/NLP-project/movie_lines.txt').read().split('\n')
convs_lines = open('/Users/troyxiao/Desktop/NLP-project/movie_conversations.txt').read().split('\n')

In [3]:
# Create a dictionary to map each line's id with its text
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [31]:
# show map result 
i=0
for k,v in id2line.items():
    i = i+1
    if(i<10):
        print(k, v)

('L479748', 'And what did you say?')
('L638646', 'Oh, yes. You noticed the way she does her hair.')
('L525290', 'Cool!')
('L638647', 'Something else. My wife, Madeleine, has several pieces of jewelry that belonged to Carlotta. She inherited them. Never wore them, they were too old-fashioned... until now. Now, when she is alone, she gets them out and looks at them handles them gently, curiously... puts them on and stares at herself in the mirror... and goes into that other world... is someone else again.')
('L345355', 'For what? Breaking my heart or ruining sex for me with any other man?')
('L345354', "I'm sorry?")
('L345357', "Forget it. I've already spent too many hours crying over you.")
('L345356', 'Uh -- both?')
('L345351', 'What?')


In [5]:
# build a list of all conversations
convs = []
for line in convs_lines:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    convs.append(_line.split(','))

print(convs[0])

['L194', 'L195', 'L196', 'L197']


In [6]:
# make Q-A pair
questions = []
answers = []

for conv in convs:
    for i in range(len(conv)-1):
        questions.append(id2line[conv[i]])
        answers.append(id2line[conv[i+1]])
        
for i in range(1, 3):
    print("Q: ", questions[i])
    print("A: ", answers[i])
    print('\n')

print("Number of questions: ", len(questions))
print("Number of answers: ", len(answers))

('Q: ', "Well, I thought we'd start with pronunciation, if that's okay with you.")
('A: ', 'Not the hacking and gagging and spitting part.  Please.')


('Q: ', 'Not the hacking and gagging and spitting part.  Please.')
('A: ', "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?")


('Number of questions: ', 221616)
('Number of answers: ', 221616)


In [7]:
# preprocess 
def filter_line(text):
    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

In [8]:
# filter out punctuation
questions_filtered = []
for question in questions:
    questions_filtered.append(filter_line(question))
    
answers_filtered = []
for answer in answers:
    answers_filtered.append(filter_line(answer))

In [9]:
#filter out string with length <3 or >20
input_texts = []
target_texts = []
for question, answer in zip(questions_filtered, answers_filtered):
    q_len = len(question.split())
    a_len = len(answer.split())
    if q_len >= 2 and q_len <= 20 and a_len >= 2 and a_len <= 20:
        input_texts.append(question)
        target_texts.append(answer)

In [10]:
for i in range(5):
    print(input_texts[i])
    print(target_texts[i])
    print('\n')

well i thought we would start with pronunciation if that is okay with you
not the hacking and gagging and spitting part  please


not the hacking and gagging and spitting part  please
okay then how about we try out some french cuisine  saturday  night


you are asking me out  that is so cute that is your name again
forget it


gosh if only we could find kat a boyfriend
let me see what i can do


c'esc ma tete this is my head
right  see  you are ready for the quiz




In [11]:
print("Number of questions: ", len(input_texts))
print("Number of answers: ", len(target_texts))

('Number of questions: ', 138332)
('Number of answers: ', 138332)


In [12]:
# Create a dictionary for the frequency of the vocabulary
vocab = {}
for question in input_texts:
    for word in question.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1
            
for answer in target_texts:
    for word in answer.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

In [13]:
# encode the data
input_vocab = {}
target_vocab = {}

for question, answer in zip(input_texts, target_texts):
    for q in question.split():
        if q not in input_vocab:
            input_vocab[q] = 1
        else:
            input_vocab[q] += 1
            
    for a in answer.split():
        if a not in target_vocab:
            target_vocab[a] = 1
        else:
            target_vocab[a] += 1

In [14]:
# Remove rare words from the vocabulary.
# We will aim to replace fewer than 5% of words with <UNK>
threshold = 10
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

In [15]:
print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)

('Size of total vocab:', 45978)
('Size of vocab we will use:', 8100)


In [16]:
# In case we want to use a different vocabulary sizes for the source and target text, 
# we can set different threshold values.
# Nonetheless, we will create dictionaries to provide a unique integer for each word.
input_word2idx = {}
word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        input_word2idx[word] = word_num
        word_num += 1
        
target_word2idx = {}
word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        target_word2idx[word] = word_num
        word_num += 1

In [17]:
# see some example
inputpairs = {k: input_word2idx[k] for k in input_word2idx.keys()[:10]}
print inputpairs

targetpairs = {k: target_word2idx[k] for k in target_word2idx.keys()[:10]}
print targetpairs

{'raining': 4011, 'yellow': 1991, 'four': 4013, 'woods': 0, "friend's": 6051, 'hanging': 1, 'prices': 3375, 'woody': 2, 'gabrielle': 1992, 'payoff': 1993}
{'raining': 4011, 'yellow': 1991, 'four': 4013, 'woods': 0, "friend's": 6051, 'hanging': 1, 'prices': 3375, 'woody': 2, 'gabrielle': 1992, 'payoff': 1993}


In [18]:
# Add the unique tokens to the vocabulary dictionaries.
codes = ['<PAD>','<EOS>','<UNK>','<GO>']

for code in codes:
    input_word2idx[code] = len(input_word2idx)+1
    
for code in codes:
    target_word2idx[code] = len(target_word2idx)+1

In [19]:
input_idx2word = dict([(idx, word) for word, idx in input_word2idx.items()])
target_idx2word = dict([(idx, word) for word, idx in target_word2idx.items()])

num_encoder_tokens = len(input_idx2word)
num_decoder_tokens = len(target_idx2word)

In [20]:
np.save('model/word-input-word2idx.npy', input_word2idx)
np.save('model/word-input-idx2word.npy', input_idx2word)
np.save('model/word-target-word2idx.npy', target_word2idx)
np.save('model/word-target-idx2word.npy', target_idx2word)

In [21]:
encoder_input_data = []

encoder_max_seq_length = 0
decoder_max_seq_length = 0

for input_words, target_words in zip(input_texts, target_texts):
    encoder_input_wids = []
    for w in input_words:
        w2idx = 1
        if w in input_word2idx:
            w2idx = input_word2idx[w]
        encoder_input_wids.append(w2idx)

    encoder_input_data.append(encoder_input_wids)
    encoder_max_seq_length = max(len(encoder_input_wids), encoder_max_seq_length)
    decoder_max_seq_length = max(len(target_words), decoder_max_seq_length)

context = dict()
context['num_encoder_tokens'] = num_encoder_tokens
context['num_decoder_tokens'] = num_decoder_tokens
context['encoder_max_seq_length'] = encoder_max_seq_length
context['decoder_max_seq_length'] = decoder_max_seq_length

np.save('model/word-context.npy', context)

In [22]:
# custom function to generate batches

def generate_batch(input_data, output_text_data):
    num_batches = len(input_data) // BATCH_SIZE
    while True:
        for batchIdx in range(0, num_batches):
            start = batchIdx * BATCH_SIZE
            end = (batchIdx + 1) * BATCH_SIZE
            encoder_input_data_batch = pad_sequences(input_data[start:end], encoder_max_seq_length)
            decoder_target_data_batch = np.zeros(shape=(BATCH_SIZE, decoder_max_seq_length, num_decoder_tokens))
            decoder_input_data_batch = np.zeros(shape=(BATCH_SIZE, decoder_max_seq_length, num_decoder_tokens))
            for lineIdx, target_words in enumerate(output_text_data[start:end]):
                for idx, w in enumerate(target_words):
                    w2idx = 0
                    if w in target_word2idx:
                        w2idx = target_word2idx[w]
                    decoder_input_data_batch[lineIdx, idx, w2idx] = 1
                    if idx > 0:
                        decoder_target_data_batch[lineIdx, idx - 1, w2idx] = 1
            yield [encoder_input_data_batch, decoder_input_data_batch], decoder_target_data_batch

In [25]:
# Compiling and training

encoder_inputs = Input(shape=(None,), name='encoder_inputs')
encoder_embedding = Embedding(input_dim=num_encoder_tokens, output_dim=HIDDEN_UNITS,
                              input_length=encoder_max_seq_length, name='encoder_embedding')
encoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, name='encoder_lstm')
encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_embedding(encoder_inputs))
encoder_states = [encoder_state_h, encoder_state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens), name='decoder_inputs')
decoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, return_sequences=True, name='decoder_lstm')
decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(decoder_inputs,
                                                                 initial_state=encoder_states)
decoder_dense = Dense(units=num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(loss='categorical_crossentropy', optimizer='adam')

json = model.to_json()
open('model/word-architecture.json', 'w').write(json)

/Users/troyxiao/.local/lib/python2.7/site-packages/keras/engine/network.py:877: UserWarning: Layer decoder_lstm was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'encoder_lstm/while/Exit_3:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'encoder_lstm/while/Exit_4:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [28]:
NUM_EPOCHS = 2

X_train, X_test, y_train, y_test = train_test_split(encoder_input_data, target_texts, test_size=0.2, random_state=42)

train_gen = generate_batch(X_train, y_train)
test_gen = generate_batch(X_test, y_test)

train_num_batches = len(X_train) // BATCH_SIZE
test_num_batches = len(X_test) // BATCH_SIZE

checkpoint = ModelCheckpoint(filepath=WEIGHT_FILE_PATH, save_best_only=True)

model.fit_generator(generator=train_gen,
                    steps_per_epoch=train_num_batches,
                    epochs=NUM_EPOCHS,
                    verbose=1,
                    validation_data=test_gen,
                    validation_steps=test_num_batches,
                    callbacks=[checkpoint])

model.save_weights(WEIGHT_FILE_PATH)

Epoch 1/2
125/864 [===>..........................] - ETA: 8:49:28 - loss: 0.5283

KeyboardInterrupt: 

In [ ]:
class ChatBot():
    """
    This is ChatBot class it takes weights for the Neural Network, compliling model
    and returns prediction in responce to input text
    """
    def __init__(self):
        """
        define all required parameters, rebuild model and load weights
        """
        self.input_word2idx = np.load('model/word-input-word2idx.npy').item()
        self.input_idx2word = np.load('model/word-input-idx2word.npy').item()
        self.target_word2idx = np.load('model/word-target-word2idx.npy').item()
        self.target_idx2word = np.load('model/word-target-idx2word.npy').item()
        context = np.load('model/word-context.npy').item()
        self.max_encoder_seq_length = context['encoder_max_seq_length']
        self.max_decoder_seq_length = context['decoder_max_seq_length']
        self.num_encoder_tokens = context['num_encoder_tokens']
        self.num_decoder_tokens = context['num_decoder_tokens']
        self.ultimate_question = 'Answer to the Ultimate Question of Life, the Universe, and Everything'

        encoder_inputs = Input(shape=(None, ), name='encoder_inputs')
        encoder_embedding = Embedding(input_dim=self.num_encoder_tokens, output_dim=HIDDEN_UNITS,
                                      input_length=self.max_encoder_seq_length, name='encoder_embedding')
        encoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, name="encoder_lstm")
        encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_embedding(encoder_inputs))
        encoder_states = [encoder_state_h, encoder_state_c]

        decoder_inputs = Input(shape=(None, self.num_decoder_tokens), name='decoder_inputs')
        decoder_lstm = LSTM(units=HIDDEN_UNITS, return_sequences=True, return_state=True, name='decoder_lstm')
        decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
        decoder_dense = Dense(self.num_decoder_tokens, activation='softmax', name='decoder_dense')
        decoder_outputs = decoder_dense(decoder_outputs)

        self.model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
        self.model.load_weights('model/word-weights.h5')
        self.model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

        self.encoder_model = Model(encoder_inputs, encoder_states)

        decoder_state_inputs = [Input(shape=(HIDDEN_UNITS,)), Input(shape=(HIDDEN_UNITS,))]
        decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)
        decoder_states = [state_h, state_c]
        decoder_outputs = decoder_dense(decoder_outputs)
        self.decoder_model = Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + decoder_states)

    def reply(self, input_text):
        """
        Takes input_text and return predicted responce
        :param input_text: string
        :return: predicted_text: string
        """
        if input_text == self.ultimate_question:
            return '42'
        input_seq = []
        input_wids = []
        for word in nltk.word_tokenize(input_text.lower()):
            idx = 1
            if word in self.input_word2idx:
                idx = self.input_word2idx[word]
            input_wids.append(idx)
        input_seq.append(input_wids)
        input_seq = pad_sequences(input_seq, self.max_encoder_seq_length)
        states_value = self.encoder_model.predict(input_seq)
        target_seq = np.zeros((1, 1, self.num_decoder_tokens))
        target_seq[0, 0, self.target_word2idx['START']] = 1
        target_text = ''
        target_text_len = 0
        terminated = False
        while not terminated:
            output_tokens, h, c = self.decoder_model.predict([target_seq] + states_value)
            sample_token_idx = np.argmax(output_tokens[0, -1, :])
            sample_word = self.target_idx2word[sample_token_idx]
            target_text_len += 1

            if sample_word != 'START' and sample_word != 'END':
                target_text += ' ' + sample_word

            if sample_word == 'END' or target_text_len >= self.max_decoder_seq_length:
                terminated = True

            target_seq = np.zeros((1, 1, self.num_decoder_tokens))
            target_seq[0, 0, sample_token_idx] = 1

            states_value = [h, c]
        return target_text.strip().replace('UNK', '')

In [ ]:
bot = ChatBot()
bot.reply('what is your name')